# Data Cleaning for Australia Data Science Jobs
In this notebook we will be cleaning the data that has been scrapped

## Importing 

In [23]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer
import fuzzywuzzy
import fuzzywuzzy as fuzz
from fuzzywuzzy import process
from tqdm.notebook import tqdm

### Defining Functions for Data Cleaning

##### Check if salary is in invalid format

In [24]:
def is_invalid(row):
    salary = ["Estimate Base Salary","Low Estimate","High Estimate"]
    is_invalid = 0
    for sal in salary:
        is_invalid += (row[sal][0] != '$') or ('.' in row[sal])
    return is_invalid > 0

##### Clean the Salary on Dataframe

In [25]:
def clean_salary(df):
    # has_salary = (df[["Estimate Base Salary","Low Estimate","High Estimate"]] != "-1").any(axis=1)
    df = df.copy()
    df = df[~df.apply(is_invalid, axis=1)]
    temp = df[["Estimate Base Salary","Low Estimate","High Estimate"]].replace({'\$':"", ",":"", "K":"000"}, regex=True).astype(int)
    df.loc[:,["Estimate Base Salary","Low Estimate","High Estimate"]] = temp
    return df

##### Clean the Rating on Dataframe 

In [26]:
def clean_rating(df):
    columns = ['Company Rating','Company Career Opportinities','Compensation and Benefits', 'Company Culture and Values',
               'Company Senior Management', 'Company Work Life Balance']
    ratings = df[columns].astype(float)
    df[columns] = ratings
    # Change empty cell with mean
    #imp_mean = SimpleImputer(strategy='mean')
    #df[columns] = imp_mean.fit_transform(ratings)
    temp = df['Companny Number of Rater'].replace({'Ratings':''}, regex=True)
    df['Companny Number of Rater'] = pd.to_numeric(temp, errors='coerce')
    
    #
    temp = df['Company Friend Reccomendation'].replace({'%':''}, regex=True)
    df['Company Friend Reccomendation'] = pd.to_numeric(temp, errors='coerce')
    temp = df['Company CEO Approval'].replace({'%':''}, regex=True)
    df['Company CEO Approval'] = pd.to_numeric(temp, errors='coerce')
    return df

##### Full Clean the Whole Dataframe

In [27]:
def full_clean(df):
    df = df.copy()
    df = clean_salary(df) # Clean the salary columns
    df.replace({'-1':None, -1:None}, inplace=True) #Replace all -1 to None 
    
    df = clean_rating(df)
    return df

##### Add Columns for Country and State

In [28]:
def add_country_state(df, country, state):
    df['Country'] = country
    df['State'] = state
    return df

In [29]:
# columns_num = ['Job Title', 'Job Location', 'Company', 'Url', 'Company Size', 'Company Type',
#                'Company Sector', 'Company Industry','Company Revenue', 'Job Descriptions']
# columns_cat = ['Company Founded']
# df2 = df.copy()
# df2.replace({'-1':None, -1:None}, inplace=True)

##### Categorizing the Job Title

In [30]:
def clean_job_titles(df):
    df = df.copy()
    column = 'Job Title'
    
    titles = ["Data Analyst", "Data Manager" ,"Data Engineer" ,"Data Scientist" ,"Data Architect" ,"Business Analyst" , "Research Scientist",
         "Data Modeller" ,"Machine Learning Engineer", "Database Administrator", "Software Engineer", "Machine Learning Scientist"]
    job_titles = df['Job Title'].unique()
    title_to_change = []
    
    for i in range(len(titles)):
        title = titles[i]
        matches = fuzz.process.extract(title, job_titles, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
        min_ratio = 77
        title_to_change.append([match[0] for match in matches if match[1] >= min_ratio])
    
    for i in range(len(titles)):
        title = titles[i]
        rows_with_matches = df[column].isin(title_to_change[i])
        df.loc[rows_with_matches, column] = title
        
    return df

## Data Cleaning

In [31]:
states = ["Victoria", "New South Wales", "Northern Territory", "Queensland",
          "South Australia", "Tasmania", "Western Australia"]

dfs = [] # List of all dataframe

# Clean all states' dataframe and add to dfs
for state in states:
    df = pd.read_csv(state+".csv")
    df = full_clean(df)
    df = add_country_state(df, "Australia", state)
    dfs.append(df)

C:\Users\nadzm\AppData\Local\Temp\ipykernel_4276\3681093875.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,["Estimate Base Salary","Low Estimate","High Estimate"]] = temp
C:\Users\nadzm\AppData\Local\Temp\ipykernel_4276\3681093875.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,["Estimate Base Salary","Low Estimate","High Estimate"]] = temp
C:\Users\nadzm\AppData\Local\Temp\ipykernel_4276\3681093875.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always s

In [32]:
full_df = dfs[0]
for i in range(1, len(dfs)):
    full_df = pd.concat([full_df, dfs[i]])
    
full_df = full_df.reset_index(drop=True)

In [33]:
job_titles = full_df['Job Title'].unique()

In [34]:
# def match(series, title):
#     print('\n' + title +':')
#     print(fuzz.process.extract(title, series, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio))

In [35]:
# titles= ["Data Analyst", "Data Manager" ,"Data Engineer" ,"Data Scientist" ,"Data Architect" ,"Business Analyst" , "Research Scientist",
#          "Data Modeller" ,"Machine Learning Engineer", "Database Administrator", "Software Engineer", "Machine Learning Scientist"]
# job_titles = df['Job Title'].unique()
# for title in titles:
#     match(job_titles, title)

In [46]:
final_df = clean_job_titles(full_df)

In [47]:
final_df = final_df.reset_index(drop=True)

In [48]:
final_df["Job Title"].value_counts()[:10]

Software Engineer                        40
Electronics Engineer                     36
Navy Careers (No Experience Required)    33
Data Analyst                             27
Consultant                               27
Engineer                                 26
Business Development Manager             25
Data Engineer                            25
Senior Channel Manager                   25
Senior Front End Engineer                23
Name: Job Title, dtype: int64

In [49]:
# final_df.to_csv("AustraliaDataScienceJob.csv", index = False)

In [58]:
def has_skills(x, skills):
    maths = ["math", "maths", "mathematic", "mathematics"]
    strings = re.split('\.|\;|\,| |\:|\n' , str(x).lower())
    return any([s in strings for s in skills])


In [59]:
def add_skills(df):
    df = df.copy()
    job_desc = df['Job Descriptions']
    skills = ["python", "r", "sql", "java", "julia", "scala", "c", "c++",
              "javascript", "spark", "hadoop", "matlab", "sas",
              "git", "excel", "nosql", "aws", "mongodb", "cassandra", 
              "hive", "hadoop", "spark", "bigml", "tableau", "powerbi", 
              "nlp", "pytorch", "tensorflow"
             ]
    for skill in skills:
        col_name = str(skill) + '_yn'
        df[col_name] = job_desc.apply(lambda x: 1 if skill in re.split('\.|\;|\,| |\:|\n' , str(x).lower()) else 0)
    
    maths = ["math", "maths", "mathematic", "mathematics"]
    df["mathematic_yn"] = job_desc.apply(lambda x:1 if has_skills(x, maths) else 0)
    
    statistics = ["stat", "stats",  "statistic", "statistics"]
    df["statistic_yn"] = job_desc.apply(lambda x:1 if has_skills(x, statistics) else 0)
    return df

In [60]:
final_df

,Job Title,Job Location,Company,Url,Estimate Base Salary,Low Estimate,High Estimate,Company Size,Company Type,Company Sector,...,Company Friend Reccomendation,Company CEO Approval,Companny Number of Rater,Company Career Opportinities,Compensation and Benefits,Company Culture and Values,Company Senior Management,Company Work Life Balance,Country,State
0,Graduate Math Teacher,Melbourne,Education and Training,https://www.glassdoor.com.au/partner/jobListin...,57511,53000,63000,1001 to 5000 Employees,School / School District,None,...,77.0,NaN,0.0,3.5,3.3,3.4,3.1,3.3,Australia,Victoria
1,Business Development Manager,Scoresby,Johnson & Johnson,https://www.glassdoor.com.au/partner/jobListin...,110373,106000,115000,10000+ Employees,Company - Public,Pharmaceutical & Biotechnology,...,87.0,91.0,NaN,3.9,4.1,4.3,3.8,4.0,Australia,Victoria
2,Analytics Consultant,Melbourne,City of Melbourne,https://www.glassdoor.com.au/partner/jobListin...,120676,114000,128000,1001 to 5000 Employees,Government,Government & Public Administration,...,59.0,53.0,5.0,2.9,3.7,3.2,2.6,3.5,Australia,Victoria
3,Data Analyst,Melbourne,Wesfarmers OneDigital,https://www.glassdoor.com.au/partner/jobListin...,140000,140000,140000,10000+ Employees,Company - Public,Retail & Wholesale,...,89.0,100.0,NaN,3.9,4.0,4.2,3.3,4.0,Australia,Victoria
4,Electronics Engineer,New South Wales,Australian Antarctic Division,https://www.glassdoor.com.au/partner/jobListin...,92076,92000,92000,501 to 1000 Employees,Government,Government & Public Administration,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Australia,Victoria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,Electronics Engineer,Western Australia,Australian Antarctic Division,https://www.glassdoor.com.au/partner/jobListin...,92076,92000,92000,501 to 1000 Employees,Government,Government & Public Administration,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Australia,Western Australia
648,Construction,Perth,Chaleen Botha,https://www.glassdoor.com.au/partner/jobListin...,55000,50000,60000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia,Western Australia
649,Construction,Perth,Chaleen Botha,https://www.glassdoor.com.au/partner/jobListin...,55000,50000,60000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australia,Western Australia
650,Metallurgical Engineer,Perth,BHP,https://www.glassdoor.com.au/partner/jobListin...,141443,121000,165000,10000+ Employees,Company - Public,"Energy, Mining, Utilities",...,82.0,88.0,NaN,3.6,4.2,3.9,3.5,3.9,Australia,Western Australia


In [61]:
transformed_df = add_skills(final_df)

In [62]:
transformed_df.sum()

C:\Users\nadzm\AppData\Local\Temp\ipykernel_4276\2105625801.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  transformed_df.sum()


Job Title                        Graduate Math TeacherBusiness Development Mana...
Job Location                     MelbourneScoresbyMelbourneMelbourneNew South W...
Company                          Education and TrainingJohnson & JohnsonCity of...
Url                              https://www.glassdoor.com.au/partner/jobListin...
Estimate Base Salary                                                      67923811
Low Estimate                                                              60054000
High Estimate                                                             77184000
Company Founded                                                           715207.0
Company Rating                                                              2093.2
Company Friend Reccomendation                                              37526.0
Company CEO Approval                                                       34630.0
Companny Number of Rater                                                      25.0
Comp

In [63]:
a = final_df["Job Descriptions"]
total = 0
for i in range(len(a)):
    b = re.split('\.|\;|\,| |\:|\n' , str(a[i]).lower())
    if "math" in b:
        total += 1

print(total)


10


In [65]:
transformed_df = add_skills(transformed_df)

In [66]:
transformed_df.to_csv("AustraliaDataScienceJob.csv", index = False)

In [67]:
transformed_df["Job Title"].value_counts()

Software Engineer                          40
Electronics Engineer                       36
Navy Careers (No Experience Required)      33
Data Analyst                               27
Consultant                                 27
                                           ..
Geophysicist                                1
Senior Customer Relations                   1
Pharmaceutical Sales Representative III     1
Mechanical Engineer                         1
Environmental Engineer                      1
Name: Job Title, Length: 181, dtype: int64

In [1]:
# Hi